CLUSTER 0


In [8]:
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import pandas as pd

phases = {
    'Phase II': ['2008-01-01', '2012-12-01'],
    'Phase III': ['2013-01-01', '2020-12-01']
}


Data=pd.read_csv("../combined_data/cluster0.csv")
Data= Data.drop(columns=["Unnamed: 0","Time","month"])
Data= Data.drop(columns=["Unnamed: 0.1"])



In [9]:
Data['Date']=pd.to_datetime(Data['Date'])

In [10]:
# Créer des DataFrames séparés pour chaque phase
phase_data = {}
for phase, (start, end) in phases.items():
    start_date = pd.to_datetime(start)
    end_date = pd.to_datetime(end)
    phase_data[phase] = Data[(Data['Date'] >= start_date) & (Data['Date'] <= end_date)]

# Afficher les premières lignes de chaque DataFrame de phase
for phase, df in phase_data.items():
    # Utiliser exec pour créer dynamiquement des variables
    exec(f"{phase.replace(' ', '_')}_df = df")

In [11]:
from statsmodels.tsa.stattools import adfuller

def test_stationarity(series, significance_level=0.05):
    result = adfuller(series.dropna())
    p_value = result[1]
    return p_value   # True si stationnaire, False sinon


In [12]:
import numpy as np

def make_sationary(data):
    # Calcul du décalage nécessaire pour rendre toutes les valeurs positives avant de prendre le logarithme
    shift_per_col = data.drop(columns=['Date']).min().abs() + 1
    data_stat = data.copy()

    # Appliquer une transformation logarithmique et des différenciations d'ordre 1 et 2 si nécessaire
    for col in data_stat.columns:
        if col != 'Date':
            data_stat[col] += shift_per_col[col]
            data_stat[col] = np.log1p(data_stat[col])
            p_value_after_log = test_stationarity(data_stat[col])  # Tester la stationnarité

            if p_value_after_log < 0.05:
                print(f"✅ Stationnaire après transformation log pour {col}.")
            else:
                # Appliquer une différenciation d'ordre 1
                data_stat[col] = data_stat[col].diff()
                p_value_after_diff1 = test_stationarity(data_stat[col])

                if p_value_after_diff1 < 0.05:
                    print(f"📉✅ Différenciation d'ordre 1 appliquée à {col} et p-value après: {p_value_after_diff1}")
                else:
                    # Appliquer une différenciation d'ordre 2
                    data_stat[col] = data_stat[col].diff()
                    p_value_after_diff2 = test_stationarity(data_stat[col])
                    print(f"📉📉 Différenciation d'ordre 2 appliquée à {col} et est stationnaire: {(p_value_after_diff2<0.05)}")
    return data_stat


In [13]:
new_phase_II_df = make_sationary(Phase_II_df)


📉✅ Différenciation d'ordre 1 appliquée à InVolReg et p-value après: 2.7264864614455285e-13
📉✅ Différenciation d'ordre 1 appliquée à OutVolReg et p-value après: 5.5526596510415915e-27
📉✅ Différenciation d'ordre 1 appliquée à InVolFin et p-value après: 5.089915845223691e-07
✅ Stationnaire après transformation log pour OutVolFin.
✅ Stationnaire après transformation log pour TVolFin.
📉✅ Différenciation d'ordre 1 appliquée à TVolReg et p-value après: 0.0037453236012960143
📉📉 Différenciation d'ordre 2 appliquée à WalletReg et est stationnaire: True
✅ Stationnaire après transformation log pour WalletFin.
📉✅ Différenciation d'ordre 1 appliquée à Price et p-value après: 3.5893556354725094e-13
📉✅ Différenciation d'ordre 1 appliquée à Coal_price_EU et p-value après: 9.119073869067885e-11
📉✅ Différenciation d'ordre 1 appliquée à Ngas_price_EU et p-value après: 0.020302345638764013
📉✅ Différenciation d'ordre 1 appliquée à Oil_price_EU et p-value après: 8.50950180784112e-05
📉✅ Différenciation d'ordr

In [14]:
for col in new_phase_II_df.drop(columns=['Date']).columns:
    p_value = test_stationarity(new_phase_II_df[col])
    if p_value < 0.05:
        print(f"La série {col} est stationnaire")
    else:
        print(f"La série {col} n'est pas stationnaire (p-value >= {0.05}).")
new_phase_II_df.to_csv("../data_stationnaires/Cluster0_Phase_II.csv")

La série InVolReg est stationnaire
La série OutVolReg est stationnaire
La série InVolFin est stationnaire
La série OutVolFin est stationnaire
La série TVolFin est stationnaire
La série TVolReg est stationnaire
La série WalletReg est stationnaire
La série WalletFin est stationnaire
La série Price est stationnaire
La série Coal_price_EU est stationnaire
La série Ngas_price_EU est stationnaire
La série Oil_price_EU est stationnaire
La série HDD est stationnaire
La série CDD est stationnaire
La série ESI est stationnaire
La série IPI est stationnaire


In [15]:
new_phase_III_df = make_sationary(Phase_III_df)

📉✅ Différenciation d'ordre 1 appliquée à InVolReg et p-value après: 2.154175134406168e-07
📉✅ Différenciation d'ordre 1 appliquée à OutVolReg et p-value après: 0.0009304495720639483
✅ Stationnaire après transformation log pour InVolFin.
✅ Stationnaire après transformation log pour OutVolFin.
✅ Stationnaire après transformation log pour TVolFin.
📉✅ Différenciation d'ordre 1 appliquée à TVolReg et p-value après: 0.0
✅ Stationnaire après transformation log pour WalletReg.
✅ Stationnaire après transformation log pour WalletFin.
📉✅ Différenciation d'ordre 1 appliquée à Price et p-value après: 2.239638087951297e-19
📉✅ Différenciation d'ordre 1 appliquée à Coal_price_EU et p-value après: 5.4025919128514003e-08
📉✅ Différenciation d'ordre 1 appliquée à Ngas_price_EU et p-value après: 2.3164798783261532e-07
📉✅ Différenciation d'ordre 1 appliquée à Oil_price_EU et p-value après: 2.3552065560241847e-09
📉✅ Différenciation d'ordre 1 appliquée à HDD et p-value après: 7.940346335293576e-05
📉✅ Différenc

c:\Users\zoero\OneDrive\Bureau\M2\S1\projet tutoré\code projet tuteuré\Banking-EU-ETS\.venv\Lib\site-packages\statsmodels\regression\linear_model.py:955: RuntimeWarning: divide by zero encountered in log
  llf = -nobs2*np.log(2*np.pi) - nobs2*np.log(ssr / nobs) - nobs2
c:\Users\zoero\OneDrive\Bureau\M2\S1\projet tutoré\code projet tuteuré\Banking-EU-ETS\.venv\Lib\site-packages\statsmodels\regression\linear_model.py:955: RuntimeWarning: divide by zero encountered in log
  llf = -nobs2*np.log(2*np.pi) - nobs2*np.log(ssr / nobs) - nobs2
c:\Users\zoero\OneDrive\Bureau\M2\S1\projet tutoré\code projet tuteuré\Banking-EU-ETS\.venv\Lib\site-packages\statsmodels\regression\linear_model.py:955: RuntimeWarning: divide by zero encountered in log
  llf = -nobs2*np.log(2*np.pi) - nobs2*np.log(ssr / nobs) - nobs2
c:\Users\zoero\OneDrive\Bureau\M2\S1\projet tutoré\code projet tuteuré\Banking-EU-ETS\.venv\Lib\site-packages\statsmodels\regression\linear_model.py:955: RuntimeWarning: divide by zero encou

In [16]:
for col in new_phase_III_df.drop(columns=['Date']).columns:
    p_value = test_stationarity(new_phase_III_df[col])
    if p_value < 0.05:
        print(f"La série {col} est stationnaire")
    else:
        print(f"La série {col} n'est pas stationnaire (p-value >= {0.05}).")
new_phase_III_df.to_csv("../data_stationnaires/Cluster0_Phase_III.csv")

c:\Users\zoero\OneDrive\Bureau\M2\S1\projet tutoré\code projet tuteuré\Banking-EU-ETS\.venv\Lib\site-packages\statsmodels\regression\linear_model.py:955: RuntimeWarning: divide by zero encountered in log
  llf = -nobs2*np.log(2*np.pi) - nobs2*np.log(ssr / nobs) - nobs2


La série InVolReg est stationnaire
La série OutVolReg est stationnaire
La série InVolFin est stationnaire
La série OutVolFin est stationnaire
La série TVolFin est stationnaire
La série TVolReg est stationnaire
La série WalletReg est stationnaire
La série WalletFin est stationnaire
La série Price est stationnaire
La série Coal_price_EU est stationnaire
La série Ngas_price_EU est stationnaire
La série Oil_price_EU est stationnaire


La série HDD est stationnaire
La série CDD est stationnaire
La série ESI est stationnaire
La série IPI est stationnaire


c:\Users\zoero\OneDrive\Bureau\M2\S1\projet tutoré\code projet tuteuré\Banking-EU-ETS\.venv\Lib\site-packages\statsmodels\regression\linear_model.py:955: RuntimeWarning: divide by zero encountered in log
  llf = -nobs2*np.log(2*np.pi) - nobs2*np.log(ssr / nobs) - nobs2
c:\Users\zoero\OneDrive\Bureau\M2\S1\projet tutoré\code projet tuteuré\Banking-EU-ETS\.venv\Lib\site-packages\statsmodels\regression\linear_model.py:955: RuntimeWarning: divide by zero encountered in log
  llf = -nobs2*np.log(2*np.pi) - nobs2*np.log(ssr / nobs) - nobs2
c:\Users\zoero\OneDrive\Bureau\M2\S1\projet tutoré\code projet tuteuré\Banking-EU-ETS\.venv\Lib\site-packages\statsmodels\regression\linear_model.py:955: RuntimeWarning: divide by zero encountered in log
  llf = -nobs2*np.log(2*np.pi) - nobs2*np.log(ssr / nobs) - nobs2


CLUSTER 1

In [17]:
Data1=pd.read_csv("../combined_data/cluster1.csv")
Data1= Data1.drop(columns=["Unnamed: 0","Time","month"])
Data1= Data1.drop(columns=["Unnamed: 0.1"])
Data1.head()

Data1['Date']=pd.to_datetime(Data1['Date'])
# Créer des DataFrames séparés pour chaque phase
phase_data = {}
for phase, (start, end) in phases.items():
    start_date = pd.to_datetime(start)
    end_date = pd.to_datetime(end)
    phase_data[phase] = Data1[(Data1['Date'] >= start_date) & (Data1['Date'] <= end_date)]

# Afficher les premières lignes de chaque DataFrame de phase
for phase, df in phase_data.items():
    # Utiliser exec pour créer dynamiquement des variables
    exec(f"{phase.replace(' ', '_')}_df = df")

In [18]:
newcluster1_phase_II_df = make_sationary(Phase_II_df)
for col in newcluster1_phase_II_df.drop(columns=['Date']).columns:
    p_value = test_stationarity(newcluster1_phase_II_df[col])
    if p_value < 0.05:
        print(f"La série {col} est stationnaire")
    else:
        print(f"La série {col} n'est pas stationnaire (p-value >= {0.05}).")
newcluster1_phase_II_df.to_csv("../data_stationnaires/Cluster1_Phase_II.csv")

📉✅ Différenciation d'ordre 1 appliquée à InVolReg et p-value après: 0.04128189042936023
📉✅ Différenciation d'ordre 1 appliquée à OutVolReg et p-value après: 0.00043657868871677834
✅ Stationnaire après transformation log pour InVolFin.
✅ Stationnaire après transformation log pour OutVolFin.
✅ Stationnaire après transformation log pour TVolFin.
📉✅ Différenciation d'ordre 1 appliquée à TVolReg et p-value après: 0.006038075664978875
✅ Stationnaire après transformation log pour WalletReg.
📉✅ Différenciation d'ordre 1 appliquée à WalletFin et p-value après: 4.308331537233137e-07
📉✅ Différenciation d'ordre 1 appliquée à Price et p-value après: 3.5893556354725094e-13
📉✅ Différenciation d'ordre 1 appliquée à Coal_price_EU et p-value après: 9.119073869067885e-11
📉✅ Différenciation d'ordre 1 appliquée à Ngas_price_EU et p-value après: 0.020302345638764013
📉✅ Différenciation d'ordre 1 appliquée à Oil_price_EU et p-value après: 8.50950180784112e-05
📉✅ Différenciation d'ordre 1 appliquée à HDD et p-

In [19]:
newcluster1_phase_III_df = make_sationary(Phase_III_df)
for col in newcluster1_phase_III_df.drop(columns=['Date']).columns:
    p_value = test_stationarity(newcluster1_phase_III_df[col])
    if p_value < 0.05:
        print(f"La série {col} est stationnaire")
    else:
        print(f"La série {col} n'est pas stationnaire (p-value >= {0.05}).")
newcluster1_phase_III_df.to_csv("../data_stationnaires/Cluster1_Phase_III.csv")

📉✅ Différenciation d'ordre 1 appliquée à InVolReg et p-value après: 0.0
📉✅ Différenciation d'ordre 1 appliquée à OutVolReg et p-value après: 1.694318306256425e-29
✅ Stationnaire après transformation log pour InVolFin.
📉✅ Différenciation d'ordre 1 appliquée à OutVolFin et p-value après: 9.088564042740936e-06
✅ Stationnaire après transformation log pour TVolFin.
📉✅ Différenciation d'ordre 1 appliquée à TVolReg et p-value après: 0.0
📉✅ Différenciation d'ordre 1 appliquée à WalletReg et p-value après: 2.5551144882620998e-08
✅ Stationnaire après transformation log pour WalletFin.
📉✅ Différenciation d'ordre 1 appliquée à Price et p-value après: 2.239638087951297e-19
📉✅ Différenciation d'ordre 1 appliquée à Coal_price_EU et p-value après: 5.4025919128514003e-08
📉✅ Différenciation d'ordre 1 appliquée à Ngas_price_EU et p-value après: 2.3164798783261532e-07
📉✅ Différenciation d'ordre 1 appliquée à Oil_price_EU et p-value après: 2.3552065560241847e-09
📉✅ Différenciation d'ordre 1 appliquée à HDD